# En-De Translation
Generate translations on WinoMT dataset with WMT19 En-De Transformer model

In [1]:
import torch

PATH="/export/data4/vzhekova/biases-data/En-De"

In [2]:
%cd $PATH

/export/data4/vzhekova/biases-data/En-De
